<a href="https://colab.research.google.com/github/rohithmsr/AI-practice/blob/main/Opik/0_intro_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Logging Traces with the Open AI Integration

In this exercise, you'll log some basic traces with Opik. You can use OpenAI or open source models via LiteLLM. You can find [the full documentation for Opik's integration with OpenAI here](https://www.comet.com/docs/opik/tracing/integrations/openai). You can find [the full documentation for Opik's integration with LiteLLM here](https://www.comet.com/docs/opik/cookbook/litellm).

# Imports & Configuration

In [1]:
%pip install opik openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.0/418.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.0 MB/s eta 0:00:00


In [7]:
import opik
import os
import getpass

# Define project name to enable tracing
os.environ["OPIK_PROJECT_NAME"] = "Logging Traces Demo"

In [3]:
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "rohithmsr" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [4]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


# Tracking OpenAI Calls

In [8]:
from opik.integrations.openai import track_openai
from openai import OpenAI

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
openai_client = track_openai(openai_client)

In [9]:
prompt="Hello, earth!"

response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role":"user", "content":prompt}
    ],
    temperature=0.69,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


# Use Open Source Models With LiteLLM
Opik also integrates with LiteLLM, which allows you to use free open-source models and supports LLM APIs from all of the major providers (Bedrock, Huggingface, VertexAI, TogetherAI, Azure, OpenAI, Groq, etc.) using the OpenAI format. [See here for a full list of LLM providers supported by LiteLLM as well as how to call them.](https://docs.litellm.ai/docs/providers)

In the following example we'll use Meta's `Llama-3.1-8B-Instruct` model hosted on the Hugging Face hub.

**If you have already run the OpenAI code above, you will need to restart your kernel before running the following code**

In [1]:
%pip install opik litellm --quiet

In [2]:
import opik
import os
import getpass

os.environ["OPIK_PROJECT_NAME"] = "Logging Traces LiteLLM"

if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········


OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


In [3]:
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data
from opik import track
import litellm

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

In [4]:
import os
import getpass

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")

Enter your Hugging Face API key: ··········


In [16]:
messages = [{ "content": "There's a llama in my garden 😱 What should I do?","role": "user"}]

response = litellm.completion(
    model="huggingface/Qwen/QwQ-32B",
    messages=messages
)

print(response.choices[0].message.content)

Okay, so there's a llama in my garden and I need to figure out what to do. First, I should stay calm. Panicking might scare the llama more. I remember that llamas are generally gentle but can be skittish. Maybe I should keep my distance to avoid stressing it. I should check if it's alone or part of a group. If it's part of a herd, there might be more llamas nearby.

I need to make sure it's not injured. If it is, I should contact animal control or a vet. But if it's healthy, maybe it's just wandered off. I should look around for any identification tags or collars. If there's a phone number, I can call the owner directly. If not, I can check with local farms or llama owners in the area. 

I should also secure my garden to prevent the llama from causing damage. Maybe close gates or put up barriers. But I shouldn't try to corner it. Maybe offer some food or water to keep it calm while I contact someone. Wait, what kind of food is safe for llamas? I think they eat hay or grass, so maybe so